# 5. Timing analysis

**Solution to the exercise proposed in the `exercise.ipynb`**

## Goal
Produce a light curve of *Mrk 421* in two energy bands, compute the ratio of the fluxes and see if there is any hint of spectral variability. For this, we will learn how to produce a light curve with Gammapy and perform different calculations over it.

## Prerequisites
Make sure to have read `README.md` in this directory with prerequisites instructions on required extra dependencies and datasets used in this exercise.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('tableau-colorblind10')

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astroquery.simbad import Simbad
from regions import CircleSkyRegion
from IPython.display import display

from gammapy.data import DataStore
from gammapy.datasets import SpectrumDataset
from gammapy.estimators import FluxPointsEstimator, LightCurveEstimator
from gammapy.estimators.utils import (
    compute_lightcurve_fvar,
    compute_lightcurve_fpp,
    compute_lightcurve_doublingtime,
    get_rebinned_axis,
    resample_energy_edges,
)
from gammapy.makers import (
    DatasetsMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel
from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True

### First load the relevant data set

In [ ]:
data_store = DataStore.from_dir("../../CTA-SDC-school")

### Set the properties of the source of interest

*Bonus*: we can use Virtual Observatory services to query some other source parameters, like its redshift.

In [ ]:
src = dict()
src['Name'] = 'Mrk 421'
src['Position'] = SkyCoord.from_name(src['Name'])

try:
    simbad = Simbad()
    simbad.add_votable_fields("rvz_redshift")
    query = simbad.query_object(src['Name'])
    src['Redshift'] = query["rvz_redshift"].data[0]
except NameError:
    print("Cannot use Simbad, will set the source redshift manually")
    src['Redshift'] = 0.030  # For Mrk 421

### Data selection
We select a subsample of data acquired on our source of interest

In [ ]:
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=src['Position'].ra,
    lat=src['Position'].dec,
    radius="3 deg",
)
selected_obs_table = data_store.obs_table.select_observations(selection)

#selection = dict(
#type="time_box",
#time_range=Time(["2006-07-29T12:00", "2006-07-30T12:00"]),
#)
#selected_obs_table = selected_obs_table.select_observations(selection)

selected_obs_table = selected_obs_table[0:50]

obs_ids = selected_obs_table["OBS_ID"]
observations = data_store.get_observations(obs_ids)

Let's create 20-min bin time intervals, for later use, and filter the observations on it:

In [ ]:
t0 = observations[0].tstart
duration = 20 * u.min
n_time_bins = int((observations[-1].tstop - t0) / duration) + 1
times = t0 + np.arange(n_time_bins) * duration
time_intervals = [Time([tstart, tstop]) for tstart, tstop in zip(times[:-1], times[1:])]
time_intervals

short_observations = observations.select_time(time_intervals)

In [ ]:
# Or just observation by observation:
#time_intervals = [Time([obs.tstart, obs.tstop]) for obs in observations]
#short_observations = observations

# Data reduction

Let's perform a 1D analysis of the data.

In [ ]:
on_region_radius = Angle("0.1 deg")

on_region = CircleSkyRegion(center=src['Position'], radius=on_region_radius)
exclusion_region = CircleSkyRegion(center=src['Position'], radius=0.5 * u.deg)
geom = WcsGeom.create(
    npix=(120, 120), binsz=0.05, skydir=src['Position'], proj="TAN", frame="icrs"
)
exclusion_mask = ~geom.region_mask([exclusion_region])

In [ ]:
energy_axis = MapAxis.from_energy_bounds(0.02, 200, nbin=5, per_decade=True, unit="TeV", name="energy")
energy_axis_true = MapAxis.from_energy_bounds(0.005, 300, nbin=10, per_decade=True, unit="TeV", name="energy_true")

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)
dataset_maker = SpectrumDatasetMaker(containment_correction=True, selection=["counts", "exposure", "edisp"])

bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)
safe_mask_maker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [ ]:
%%time

# Parallel version
makers = [dataset_maker, bkg_maker, safe_mask_maker]  # the order matters
datasets_maker = DatasetsMaker(makers, stack_datasets=False, n_jobs=6)
datasets = datasets_maker.run(dataset_empty, observations)

In [ ]:
dataset_stack = datasets.stack_reduce()

Let's find some energy at which to slice our data, to build two well-balanced sets into two energy bands:

In [ ]:
excess = dataset_stack.excess.data.T[0][0]
mask = excess > 0
excess = excess[mask]
energy_bins = dataset_stack.counts.geom.axes['energy'].center[mask]
split_value = 2./3. * np.sum(excess)
split_mask = np.cumsum(excess) > split_value
e_split = energy_bins[split_mask][0]

print(f'Split energy: {e_split:.3f}')

In [ ]:
e_min = dataset_stack.energy_range_safe[0].data[0][0] * dataset_stack.energy_range_safe[0].unit
e_max = dataset_stack.energy_range_safe[-1].data[0][0] * dataset_stack.energy_range_safe[0].unit
print(f'Energy threshold: {e_min:.3f}')
print(f'Maximal energy: {e_max:.3f}')

# Fit overall spectrum

In [ ]:
spectral_model = PowerLawSpectralModel(
    amplitude=1.0e-11 * u.Unit("cm-2 s-1 TeV-1"),
    reference=e_split,
    index=2.
)
spectral_model.parameters["amplitude"].frozen = False
spectral_model.parameters["index"].frozen = False

In [ ]:
source_skymodel = SkyModel(
    spectral_model=spectral_model,
    name=src["Name"]
)

for dataset in datasets:
    dataset.models = source_skymodel

In [ ]:
%%time

fit_joint = Fit()
result_joint = fit_joint.run(datasets=datasets)

# We make a copy here of the optimised model for later use
model_best_joint = source_skymodel.copy(name=src["Name"])

In [ ]:
print(result_joint)
display(result_joint.models.to_parameters_table())

In [ ]:
### Compute SED flux points

In [ ]:
%%time

energy_edges = resample_energy_edges(dataset_stack, conditions={'sqrt_ts_min': 2.})

fpe = FluxPointsEstimator(
    energy_edges=energy_edges,
    source=src["Name"],
    selection_optional=["errn-errp", "ul", "scan"],
    fit=fit_joint,
    n_jobs=6,
)
flux_points = fpe.run(datasets)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

plot_kwargs = {
    "energy_bounds": [e_min, e_max],
    "sed_type": "e2dnde",
    "yunits": u.Unit("erg cm-2 s-1"),
    "ax": ax,
}

# plot joint model
model_best_joint.spectral_model.plot(
    **plot_kwargs, ls="--"
)
model_best_joint.spectral_model.plot_error(facecolor="grey", alpha=0.7, **plot_kwargs)

flux_points.plot(ax=plot_kwargs["ax"],
                 sed_type=plot_kwargs["sed_type"],
                 )

# Light curve

Let's compute the light curve in two energy ranges

In [ ]:
energy_lo = [e_min.value, e_split.value] * e_min.unit
energy_hi = [e_split.value, e_max.value] * e_max.unit
energy_all = [e_min.value, e_max.value] * e_min.unit

e_ranges = [energy_lo, energy_hi, energy_all]

In [ ]:
lc_maker = LightCurveEstimator(
    energy_edges=[e_min, e_split, e_max],
    source=model_best_joint.name,
    time_intervals=time_intervals,
    selection_optional=["errn-errp", "ul", "scan"],
    n_jobs=6,
)
lc = lc_maker.run(datasets)

In [ ]:
lc.plot(sed_type="flux", time_format="mjd", axis_name="time")

We can also rebin the light curve to larger, fixed time bins, or requesting a minimum TS:

In [ ]:
lc_maker_new = LightCurveEstimator(
    energy_edges=[e_min, e_max],
    source=model_best_joint.name,
    time_intervals=time_intervals,
    selection_optional="all",
    n_jobs=6,
)
lc_all = lc_maker_new.run(datasets)

In [ ]:
# axis_new = get_rebinned_axis(lc_all, method="min-ts", ts_threshold=3000, axis_name="time")
axis_new = get_rebinned_axis(lc_all, method="fixed-bins", group_size=3, axis_name="time")
print(axis_new)

In [ ]:
lc_new = lc_all.resample_axis(axis_new)

plt.figure(figsize=(8, 6))
ax = lc_all.plot(label="original")
lc_new.plot(ax=ax, label="rebinned")
plt.legend()
plt.show()

# Fractional and point-to-point variability

In [ ]:
compute_lightcurve_fvar(lc, flux_quantity='flux').pprint_all()

In [ ]:
compute_lightcurve_fpp(lc, flux_quantity='flux').pprint_all()

In [ ]:
compute_lightcurve_doublingtime(lc, flux_quantity='flux').pprint_all()

# Hardness ratio diagrams

Let's compute the flux ratio of our two light curves, plot them against time, and against the overall flux (i.e. hardness ratio diagram).

Access the low-energy and high-energy light curves.

**Tip**: Remember that `RegionNDMap` holds quantities of `numpy.ndarray`s.

In [ ]:
lc_lo = lc.flux.quantity[:, [0]].flatten()
lc_lo_err = lc.flux_err.quantity[:, [0]].flatten()

lc_hi = lc.flux.quantity[:, [1]].flatten()
lc_hi_err = lc.flux_err.quantity[:, [1]].flatten()

lc_tot = lc_lo+lc_hi
lc_tot_err = np.sqrt(lc_hi_err**2 + lc_lo_err**2)

In [ ]:
flux_ratio = lc_hi/lc_lo

In [ ]:
flux_ratio_err = flux_ratio * np.sqrt((lc_hi_err/lc_hi)**2+(lc_lo_err/lc_lo)**2)

In [ ]:
plt.errorbar(x=lc.geom.axes["time"].time_mid.mjd, y=flux_ratio, yerr=flux_ratio_err, fmt='o')
plt.xlabel("Time (MJD)")
plt.ylabel(
    f"Hardness ratio ({e_ranges[1][0]:.1f}-{e_ranges[1][1]:.1f}/{e_ranges[0][0]:.1f}-{e_ranges[0][1]:.1f})"
)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

t = ((lc.geom.axes["time"].time_mid.mjd-lc.geom.axes["time"].time_mid.mjd[0]) * u.d).to(u.h)

cmap_norm = matplotlib.colors.Normalize(vmin=min(t.value), vmax=max(t.value))
mapper = matplotlib.cm.ScalarMappable(norm=cmap_norm, cmap='cividis')
time_colors = np.array([(mapper.to_rgba(_)) for _ in t.value])

for _x, _y, _ex, _ey, _color in zip(flux_ratio, lc_tot, flux_ratio_err, lc_tot_err, time_colors):
    ax.plot(_x, _y, 'o', color=_color)
    ax.errorbar(
        x=_x,
        xerr=_ex,
        y=_y,
        yerr=_ey,
        color=_color,
    )
ax.set_xlabel(
    f"Hardness ratio ({e_ranges[1][0]:.1f}-{e_ranges[1][1]:.1f}/{e_ranges[0][0]:.1f}-{e_ranges[0][1]:.1f})"
)
ax.set_ylabel(f"Flux ({e_ranges[2][0]:.1f}-{e_ranges[2][1]:.1f})")
fig.colorbar(mapper, ax=ax, orientation='vertical', label="Time (h)")